# Evaluation of uncertainty of models on 60km -> 2.2km-4x over Birmingham

In [ ]:
%reload_ext autoreload

%autoreload 2

%reload_ext dotenv
%dotenv

import functools
import math
import string

import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from properscoring import crps_ensemble
import xarray as xr

from mlde_notebooks.data import prep_eval_data
from mlde_notebooks.display import pretty_table
from mlde_notebooks.uncertainty import plot_spread_error, plot_domain_means

import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import seaborn as sns
import xarray as xr
import xskillscore as xs

from mlde_utils import cp_model_rotated_pole, dataset_split_path
from mlde_notebooks import plot_map
from mlde_notebooks.data import open_dataset_split, attach_eval_coords, attach_derived_variables
from mlde_notebooks.display import pretty_table
from mlde_notebooks.distribution import xr_hist, hist_dist
from mlde_notebooks.examples import em_timestamps

In [ ]:
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from mlde_notebooks.default_params import *

In [ ]:
IPython.display.Markdown(desc)

In [ ]:
EVAL_DS, MODELS = prep_eval_data(sample_configs, dataset_configs, derived_variables_config, eval_vars, split, ensemble_members=ensemble_members, samples_per_run=samples_per_run)
EVAL_DS

In [ ]:
CPM_DAS = { var: EVAL_DS["CPM"][f"target_{var}"] for var in eval_vars }

## Figure: spread-error variability

* Spread-error
  * https://journals.ametsoc.org/view/journals/hydr/15/4/jhm-d-14-0008_1.xml?tab_body=fulltext-display
  * https://journals.ametsoc.org/view/journals/aies/2/2/AIES-D-22-0061.1.xml
  * https://www.sciencedirect.com/science/article/pii/S0021999107000812

In [ ]:
def resample_examples(da):
    return xs.resampling.resample_iterations_idx(da.stack(member=["ensemble_member", "time"]), 10, "member", replace=True).unstack("member")

bs_ds = EVAL_DS["CPM"][[var_name for var in eval_vars for var_name in [f"target_{var}", f"pred_{var}"]]].groupby("stratum").map(resample_examples)

bs_ds

In [ ]:
for var in eval_vars:
    ss_fig = plt.figure(layout='constrained', figsize=(4, 2))
    axd = ss_fig.subplot_mosaic(bs_ds.model.values.reshape(1, -1))

    for model, model_ds in bs_ds.groupby("model", squeeze=False):
        ax= axd[model]
        for _, iteration_ds in model_ds.groupby("iteration"):
            mois = { model: {"color": None} }
            pred_da = iteration_ds[f"pred_{var}"]
            target_da = iteration_ds[f"target_{var}"]
            plot_spread_error(pred_da=pred_da, target_da=target_da, ax=ax, line_props=mois)
        ax.set_title(f"{model}")
    ss_fig.suptitle(var)
        
    plt.show()